In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: Coronary CT Angiography.pdf
  ✓ Loaded 12 pages

Total documents loaded: 12


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-10-09T19:09:53+05:30', 'author': 'HP', 'moddate': '2025-10-09T19:09:53+05:30', 'source': '..\\data\\pdf\\Coronary CT Angiography.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1', 'source_file': 'Coronary CT Angiography.pdf', 'file_type': 'pdf'}, page_content='Coronary CT Angiography (CCTA) & Heart Attack: Comprehensive Reference \n \nTable of Contents \n1. Introduction \n2. Anatomy & Physiology of Coronary Circulation \n3. Pathophysiology of Coronary Artery Disease & Myocardial Infarction \n4. What is CCTA? \n5. Technical Principles of CCTA \n1. CT Hardware & Detectors \n2. ECG Gating (Prospective, Retrospective) \n3. Contrast Use & Protocols \n4. Image Acquisition & Reconstruction \n5. Post-processing & Quantitative Analysis \n6. Clinical Indications & Guidelines for CCTA \n7. Interpretation of CCTA Findings \n1. Stenosis Grading \n2. Plaque Characterization & Vulnerab